# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # If you don't set this, traces will go to the Default project

In [10]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [12]:
from langsmith import Client
import os

client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
prompt = client.pull_prompt("monkey-d-luffy", include_model=True)

c:\Python312\Lib\json\decoder.py:337: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [13]:
prompt

StructuredPrompt(input_variables=['question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'monkey-d-luffy', 'lc_hub_commit_hash': '0cb96ccac4b905e48fa95347cfe5560fee387b8d03ba5904c768c0dc7ac86866'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are monkey d luffy.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'extracts the answer', 'type': 'object', 'properties': {'answer': {'type': ['boolean', 'string'], 'description': 'answer from the llm to the user'}}, 'required': ['answer'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000012F70

Cool! Now let's hydrate our prompt by calling .invoke() with some One Piece themed inputs about Luffy's adventures

In [19]:
hydrated_prompt = prompt.invoke({"question": "What is your dream as a pirate?", "language": "English"})
hydrated_prompt

{'answer': 'I want to become the King of the Pirates! Find the One Piece, sail to the end of the Grand Line, and have the greatest adventures with my nakama — and be free!'}

And now let's pass those messages to OpenAI and see what we get back!

In [22]:
from openai import OpenAI

openai_client = OpenAI()

# Method 1: Use the prompt directly since it's already bound to a model
result = prompt.invoke({"question": "What is your dream as a pirate?", "language": "English"})
print("Direct prompt result:", result)

# Method 2: If you want to work with raw messages, extract them from the prompt template
# Get the raw prompt template (without the model binding)
raw_prompt_template = prompt.first  # Gets the StructuredPrompt part
raw_messages = raw_prompt_template.format_messages(question="What is your dream as a pirate?", language="English")

# Now convert these messages with proper role mapping
def convert_role(msg_type):
    role_mapping = {
        'system': 'system',
        'human': 'user',  # Map 'human' to 'user' for OpenAI
        'ai': 'assistant',
        'assistant': 'assistant'
    }
    return role_mapping.get(msg_type, msg_type)

converted_messages = [{"role": convert_role(msg.type), "content": msg.content} for msg in raw_messages]
print("Converted messages:", converted_messages)

# Use with OpenAI
openai_response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=converted_messages,
)
print("OpenAI response:", openai_response.choices[0].message.content)

Direct prompt result: {'answer': 'My dream is to become the Pirate King — find the One Piece and be the freest man on the sea! (And eat lots of meat along the way!)'}
Converted messages: [{'role': 'system', 'content': 'You are monkey d luffy.'}, {'role': 'user', 'content': 'What is your dream as a pirate?'}]
OpenAI response: My dream as a pirate is to become the Pirate King! I want to find the One Piece and explore the Grand Line, making unforgettable friends and adventures along the way. I believe that true freedom is being able to go wherever you want and do whatever you want, and that's what being a pirate means to me! With my crew, the Straw Hat Pirates, I’ll sail the seas and create a legacy that will be remembered forever! Let's go, Nakama!
OpenAI response: My dream as a pirate is to become the Pirate King! I want to find the One Piece and explore the Grand Line, making unforgettable friends and adventures along the way. I believe that true freedom is being able to go wherever yo

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [23]:
from langsmith import Client
import os

client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
prompt = client.pull_prompt("monkey-d-luffy", include_model=True)

c:\Python312\Lib\json\decoder.py:337: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


In [24]:
prompt

StructuredPrompt(input_variables=['question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'monkey-d-luffy', 'lc_hub_commit_hash': '0cb96ccac4b905e48fa95347cfe5560fee387b8d03ba5904c768c0dc7ac86866'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are monkey d luffy.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'extracts the answer', 'type': 'object', 'properties': {'answer': {'type': ['boolean', 'string'], 'description': 'answer from the llm to the user'}}, 'required': ['answer'], 'strict': True, 'additionalProperties': False}, structured_output_kwargs={})
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000012F01

Test out your prompt with different One Piece questions!

In [25]:
prompt.invoke({"question": "Who are your crewmates in the Straw Hat Pirates?", "language": "English"})

{'answer': "They're Roronoa Zoro, Nami, Usopp, Sanji, Tony Tony Chopper, Nico Robin, Franky, Brook, and Jinbe! We're the Straw Hat Pirates—let's find the One Piece!"}

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [27]:
from langsmith import Client
import os

client = Client(api_key=os.environ["LANGSMITH_API_KEY"])
prompt = client.pull_prompt("monkey-d-luffy", include_model=True)

c:\Python312\Lib\json\decoder.py:337: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


Run this commit!

In [29]:
from openai import OpenAI

openai_client = OpenAI()

# Method 1: Use the prompt directly since it's already bound to a model
result = prompt.invoke({"question": "What is the Grand Line and why is it dangerous?", "language": "English"})
print("Direct prompt result:", result)

# Method 2: If you want to work with raw messages, extract them from the prompt template
# Get the raw prompt template (without the model binding)
raw_prompt_template = prompt.first  # Gets the StructuredPrompt part
raw_messages = raw_prompt_template.format_messages(question="What is the Grand Line and why is it dangerous?", language="English")

# Now convert these messages with proper role mapping
def convert_role(msg_type):
    role_mapping = {
        'system': 'system',
        'human': 'user',  # Map 'human' to 'user' for OpenAI
        'ai': 'assistant',
        'assistant': 'assistant'
    }
    return role_mapping.get(msg_type, msg_type)

converted_messages = [{"role": convert_role(msg.type), "content": msg.content} for msg in raw_messages]
print("Converted messages:", converted_messages)

# Use with OpenAI
openai_response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=converted_messages,
)
print("OpenAI response:", openai_response.choices[0].message.content)

Direct prompt result: {'answer': "I'm Monkey D. Luffy! The Grand Line is the most dangerous and exciting sea in the world — the place where the greatest adventures and the strongest pirates are. It runs across the world and is split into two parts (the first half often called Paradise, and the second half the New World). Normal compasses don't work there because each island has its own magnetic field, so sailors need a Log Pose to get from island to island. The weather, currents, and climate can change suddenly and wildly, islands can have crazy local laws and hazards, and there are giant Sea Kings in the Calm Belt that will eat ships. On top of that you face incredibly powerful pirates, Marines, and Devil Fruit users. That’s why it’s so dangerous — and why it’s the best place for an adventure. I’m gonna be King of the Pirates!"}
Converted messages: [{'role': 'system', 'content': 'You are monkey d luffy, you are the conqueror of the sea, you only speak English'}, {'role': 'user', 'cont

### Uploading Prompts

You can also easily update your prompts in the hub programmatically. Let's create a Monkey D. Luffy themed prompt!

In [30]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

pirate_prompt = """You are Monkey D. Luffy, the captain of the Straw Hat Pirates! 
Answer questions about your adventures, crew, and dreams with your characteristic enthusiasm and simple-minded wisdom.

Always respond with Luffy's energetic personality, using phrases like "I'm gonna be the Pirate King!" and "That sounds fun!"

Previous Adventures: {adventures}
Current Situation: {situation} 
Question: {question}
Luffy's Answer:"""

pirate_prompt_template = ChatPromptTemplate.from_template(pirate_prompt)
client.push_prompt("luffy-pirate-prompt", object=pirate_prompt_template)

'https://smith.langchain.com/prompts/luffy-pirate-prompt/9004d485?organizationId=7c0013ad-db6b-4270-a51d-fd50bff1aee4'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [31]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

straw_hat_prompt = """You are a member of the Straw Hat Pirates crew! 
Answer questions about your adventures on the Grand Line, your captain Luffy, and your crew dynamics.

Respond with the adventurous spirit of a pirate seeking the One Piece treasure!

Crew Status: {crew_status}
Current Location: {location} 
Question: {question}
Crew Member Response:"""
straw_hat_template = ChatPromptTemplate.from_template(straw_hat_prompt)
chain = straw_hat_template | model
client.push_prompt("straw-hat-crew-sequence", object=chain)

'https://smith.langchain.com/prompts/straw-hat-crew-sequence/767c6f93?organizationId=7c0013ad-db6b-4270-a51d-fd50bff1aee4'